# Module 5: Web Scraping Using `BeautifulSoup`
## DS 6001: Practice and Application of Data Science

**Table of contents**:

* [Introduction: Is Web Scraping Legal?](#legal)
* [How Websites Prevent You From Scraping](#prevent)
* [Using `requests` with a User Agent Header](#useragent)
* [Using `BeautifulSoup()` (Example: WNRN, Charlottesville's Legendary Radio Station)](#beausoup)
    * [Downloading and Understanding Raw HTML](#html)
    * [Parsing Raw HTML with `BeautifulSoup()`](#parse)
    * [Searching for HTML Tags and Extracting Data](#tags)
    * [Constructing a Data Frame from HTML Data](#dataframe)
* [Building a Spider](#spider)

## <a name="legal"></a> Introduction: Is Web Scraping Legal?
Web scraping is the practice of downloading the raw HTML code that generates a website, and instead of parsing the code to display it like we do with a web browser whenever we navigate to a page, we dig through the code for data. 

There are **significant legal and ethical questions** regarding web scraping. Right now, there is a court case - [HiQ Labs vs. LinkedIn](https://en.wikipedia.org/wiki/HiQ_Labs_v._LinkedIn) - that will establish whether and under what conditions web scraping is legal. 

<img src="https://crowdjustice.imgix.net/pictures/courtroom5-gray_uI7BCRV.png?auto=enhance%2Cformat&crop=faces&fit=crop&q=80&w=1200" width="600" height="600">

HiQ is a company that scraps data from individuals' LinkedIn profiles to compile data to use to build models that predict whether employees will leave their positions for new jobs. They use the insights from these models to consult with businesses to reduce employee turnover. In 2017 LinkedIn issued a Cease and Desist order to HiQ and took steps to prevent anyone including HiQ from deploying scrapers on profiles. Because HiQ's business model is entirely dependent on collecting data from LinkedIn profiles, this action by LinkedIn would have caused HiQ to go out of business. Instead, HiQ filed a lawsuit against LinkedIn in federal court. HiQ argued that user profiles contain data that are owned by the LinkedIn users themselves who intend to make that information public using the LinkedIn platform -- and so, because the data owners are sharing their own data, HiQ should be able to access that data. LinkedIn argued that the HiQ's scraping of user data amounts to theft, is a violation of the [Consumer Fraud and Abuse Act](https://en.wikipedia.org/wiki/Computer_Fraud_and_Abuse_Act), and violates LinkedIn users' privacy. 

In 2017 a federal district court issued an injunction that prevented LinkedIn from blocking HiQ's scraping scripts. The case was appealed to the Circuit Court, where [this injuction was upheld in 2019](https://www.reuters.com/article/us-microsoft-linkedin-profiles/microsofts-linkedin-loses-appeal-over-access-to-user-profiles-idUSKCN1VU21W). In March 2020 LinkedIn filed for the case to be appealed to the U.S. Supreme Court. It is likely that this case will be decided at the highest level in the United States sometime in the next year. What the Supreme Court decides will have [massive ramifications](https://www.natlawreview.com/print/article/linkedin-files-petition-to-supreme-court-hiq-web-scraping-case) on the use of web scraping, data science, and on the tech industry at large. It will establish whether web scraping is legal, legal with restrictions, or illegal, and it will have implications for data ownership and the extent to which large tech companies control and own the data that exist on their platforms.

If this debate is interesting to you, here's a longer, philosophically-oriented article about the virtues of web scraping for academic research: https://research.gold.ac.uk/6768/1/Marres_Weltrevede_Scraping_the_Social_draft.pdf

Even if scraping is legal, it might not be ethical. Scraping calls a server that is not designed for the transference of data the way an API is. As such, many repeated calls (from bots) to a website for the purpose of scraping data might overwhelm the server, keeping the website's owners from being able to achieve their purpose for having the website. This [article by James Densmore](https://towardsdatascience.com/ethics-in-web-scraping-b96b18136f01) outlines some ethical considerations for scrapers and for website owners, and lays out the following code of conduct: 

"I, the web scraper will live by the following principles:

* If you have a public API that provides the data I’m looking for, I’ll use it and avoid scraping all together.

* I will always provide a User Agent string that makes my intentions clear and provides a way for you to contact me with questions or concerns.

* I will request data at a reasonable rate. I will strive to never be confused for a DDoS attack.

* I will only save the data I absolutely need from your page. If all I need it OpenGraph meta-data, that’s all I’ll keep.

* I will respect any content I do keep. I’ll never pass it off as my own.

* I will look for ways to return value to you. Maybe I can drive some (real) traffic to your site or credit you in an article or post.

* I will respond in a timely fashion to your outreach and work with you towards a resolution.

* I will scrape for the purpose of creating new value from the data, not to duplicate it.

 . . . . I, the site owner will live by the following principles:

* I will allow ethical scrapers to access my site as long as they are not a burden on my site’s performance.

* I will respect transparent User Agent strings rather than blocking them and encouraging use of scrapers masked as human visitors.

* I will reach out to the owner of the scraper (thanks to their ethical User Agent string) before blocking permanently. A temporary block is acceptable in the case of site performance or ethical concerns.

* I understand that scrapers are a reality of the open web.

* I will consider public APIs to provide data as an alternative to scrapers."

In general, while scraping appears to exist in a legal and ethical grey area, there is a lot of litigation and many arguments that suggest that once information is posted publically on a website, that there is no longer an expectation of privacy, and the data is open for anyone to use. But be careful when you use web scraping! If there IS an expectation of privacy, such as for any website with a password or other restrictions on access, then the legality and ethics of web scraping are much more dubious. If an API is available, it's available for a reason, and it's the correct ethical decision to use the API instead of scraping. Use care and judgment.

## <a name="prevent"></a> How Websites Prevent You From Scraping
This discussion follows the excellent overview by a Stack Overflow and GitHub contributor with the username JonasCz (I wish I knew this user's real name!) on [how to prevent web scraping](https://github.com/JonasCz/How-To-Prevent-Scraping/blob/master/README.md).

To understand the restrictions and challenges you will encounter when scraping data, put yourself in the position of a website's owner:

If you own and maintain a website, there are many reasons why you might want to prevent web scraping bots from accessing the data on your website. Maybe the bots will overload the traffic to your site and make it impossible for your website to work as you intend. You might be running a business through this website and sharing the data in mass transfers would undercut your business. For whatever reason, you are now faced with a challenge: how to you prevent automated scraping of the data on your webpage while still allowing individual customers to view your website?

Web scraping will require issuing HTTP requests to a particular web address with a tool like `requests`, sometimes many times in a short period. Every HTTP request is logged by the server that receives the request, and these logs contain the IP address of the entity making the request. If too many requests are made by the same IP address, the server can block that IP address. The coding logic to automatically identify and block overactive IP addresses is simple, so many websites include these security measures. Some blocks are temporary, placing a rate limit on these requests to slow down the scrapers, and some blocks reroute scrapers through a CAPTCHA (which stands for "Completely Automated Test to Tell Computers and Humans Apart") to prevent robots like a scraper from accessing the website. JonasCz recommends that these security measures look at other factors as well: the speed of actions on the website, the amount of data requested, and other factors that can identify a user when the IP address is masked.

Stronger gates, such as making users register for a username and password with email confirmation to use your website, are effective against scraping bots. But they also turn away individuals who wouldn't want to jump through those hoops. Saving all text as images on your server will prevent bots from accessing the text very easily, but it makes the website harder to use and violates regulations that protect people with disabilities.

Instead, JonasCz recommends building your website in a way that never reveals the entirety of the data you own, and never reveals the private API endpoints you use to display the data. Also, web scrapers are fragile: they are built to pull data from the specific HTML structure of a particular website. Changing the HTML code frequently or using different versions of the code based on geographic location will break the scrapers that are built for that code. JonasCz also suggests adding "honeypot" links to the HTML code that will not be displayed to legitimate users but will be followed by scrapers that recursively follow links, and taking action against the agents that follow these links: block their IP addresses, require a CAPTCHA, or deliver fake data.

One important piece of information in a request is the user agent header (which we discuss in more detail [below](#useragent)). JonasCz recommends looking at this information and blocking requests when the user agent is blank or matches information from agents that have previously been identified as malicious bots.

Understanding the steps you would take to protect your data from bots if you owned a website, you should have greater insight into why a web scraping endeavor may fail. Your web scraper might not be malicious, but might still violate the rules that the website owner setup to guard against bots. These rules are usually listed explicitly in a file on the server, usually called `robots.txt`. Some tips for reading and understanding a `robots.txt` file are here: https://www.promptcloud.com/blog/how-to-read-and-respect-robots-file/

For example, in this document we will be scraping data on the playlist of a radio station from https://spinitron.com/. This website has a `robots.txt` file here: https://spinitron.com/robots.txt, which reads:
```
User-agent: *
Crawl-delay: 10
Request-rate: 1/10
```
The `User-agent: *` line tells us that the next two lines apply to all user agent strings. `Crawl-delay: 10` places a limit on the frequency with which our scraper can make a request from this website. In this case, individual requests must be made 10 second apart. `Request-rate: 1/10` tells us that our scraper is only allowed to access one page every 10 seconds, and that we are not allowed to make requests from more than one page at the same time.

## <a name="useragent"></a> Using `requests` with a User Agent Header
As the articles by James Densmore and JonasCz described, requests are much more likely to get blocked by websites if the request does not specify a header that contains a user agent. An HTTP header is a parameter that gets sent along with the HTTP request that contains metadata about the request. A user agent header contains contact and identification information about the person making the request. If there is any issue with your web scraper, you want to give the website owner a chance to contact you directly about that problem. If you do not feel comfortable being contacted by the website's owner, you should reconsider whether you should be scraping that website.

Fortunately, it is straightforward to include headers in a GET request using `requests`: just use the `headers` argument. First, we import the relevant libraries:

In [1]:
import numpy as np
import pandas as pd
import requests

In module 4 we issued GET requests from the Wikipedia API as an example.

In [2]:
r = requests.get("https://en.wikipedia.org/w/api.php")
r

<Response [200]>

To add a user agent string, I use the following code:

In [3]:
headers = {'user-agent': 'Kropko class example (jkropko@virginia.edu)'}
r = requests.get("https://en.wikipedia.org/w/api.php", headers = headers)
r

<Response [200]>

What information needs to go into a user agent header? Different resources have different information about that. According to [Amazon Web Services](https://docs.developer.amazonservices.com/en_US/dev_guide/DG_UserAgentHeader.html), a user agent should identify your application, its version number, and programming language. So a user agent should look like this:

In [4]:
headers = {'user-agent': 'Kropko class example version 1.0 (jkropko@virginia.edu) (Language=Python 3.8.2; Platform=Mac OSX 10.15.5)'}
r = requests.get("https://en.wikipedia.org/w/api.php", headers = headers)
r

<Response [200]>

Including a user agent is not hard, and it goes a long way towards alleviating the anxieties that website owners have about dealing with your web scraping code. It is a good practice to cultivate into a habit. 

## <a name="beausoup"></a> Using `BeautifulSoup()` (Example: WNRN, Charlottesville's Legendary Radio Station)
WNRN is a legendary radio station, and it's based right here in Charlottesville at 91.9 FM (and streaming online at www.wnrn.org). It's commercial-free, with only a few interruptions for local nonprofits to tell you about cool things happening in town. They play a mix of new and classic alternative rock and R&B. They emphasize music for bands coming to play at local venues. And they play the Grateful Dead on Saturday mornings. You should be listening to WNRN!

<center><img src="https://upload.wikimedia.org/wikipedia/en/thumb/0/0e/WNRN-FM_2014.PNG/200px-WNRN-FM_2014.PNG" width="300" height="300" class="center"></img></center>

The playlist of the songs that WNRN has played in the last few hours is here: https://spinitron.com/WNRN/. I want to scrape the data off this website. I also want to scrape the data off of the additional playlists that this website links to, to collect as much data as possible. Our goal in this example is to create a dataframe of each song WNRN has played, the artist, the album, and the time each song was played.

The process involves four steps:

1. Download the raw text of the HTML code for the website we want to scrape using the `requests` library.

2. Use the `BeautifulSoup()` function from the `bs4` library to parse the raw text so that Python can understand, search through, and operate on the HTML tags from string.

3. Use methods associated with `BeautifulSoup()` to extract the data we need from the HTML code.

4. Place the data into a `pandas` data frame.

### <a name="html"></a> Downloading and Understanding Raw HTML
For this example, I first download the HTML that exists on https://spinitron.com/WNRN using the `requests.get()` function. To be ethical and to help this website's owners know that I am not a malicious actor, I also specify a user agent string. 

In [5]:
url = "https://spinitron.com/WNRN"
headers = {'user-agent': 'Kropko class example (jkropko@virginia.edu)'}
r = requests.get(url, headers=headers)
r

<Response [200]>

The raw HTML code contains a series of text fragments that look like this,
```
<tag attribute="value"> Navigable string </tag>
```
where `tag`, `attribute`, `"value"`, and `Navigable string` are replaced by specific parameters and data that control the content and presentation of the webpage that gets displayed in a web browser. For example, here are the first 1000 characters of the raw text from WNRN's playlist:

In [33]:
print(r.text[0:1000])


<!doctype html><html lang="en">
<head>
    <meta charset="utf-8">
    <meta http-equiv="X-UA-Compatible" content="IE=edge">
    <meta name="viewport" content="width=device-width, initial-scale=1,maximum-scale=1">
    <title>WNRN – Independent Music Radio</title>
    <meta name="description" content="A member supported Independent Music Radio Station serving Central Virginia and the Shenandoah Valley">

    <link rel="apple-touch-icon" href="apple-touch-icon.png">
                <meta name="csrf-param" content="_csrf">
<meta name="csrf-token" content="2dDjfP_QL-ckC4pxxRHzuVUt_k5hCg14icxrhuPZZsewlIcQnL1-kBd64SKSIpnIMxmfPFVpRTvaniDpj4pXrA==">

    <meta property="og:url" content="/WNRN/">
<meta property="og:title" content="WNRN – Independent Music Radio">
<meta property="og:description" content="A member supported Independent Music Radio Station serving Central Virginia and the Shenandoah Valley">
<meta property="og:image" content="https://spinitron.com/images/Station/13/1326-img_logo.2

**Tags** specify how the data contained within the page are organized and how the visual elements on this page should look. Tags are designated by opening and closing angle braces, < and >. In the HTML code displayed above, there are tags named 

* `<html>`, which tells browsers that the following code is written in HTML, 
* `<meta>`, which defines metadata in the document that help govern how the output shold be displayed in the browser, 
* `<title>`, which sets the title of the document, and 
* `<link>`, which pulls data or images from external resources for later use. 

To see what other HTML tags do, look at the list on https://www.w3schools.com/TAGs/. 

In some cases the tag operates on the text that immediately follows, and a closing tag `</tag>` frames the text that gets operated on by the tag. The text in between the opening and closing tag is called the **navigable string**. For example, the tag `<title>WNRN – Independent Music Radio</title>` specifies that "WNRN – Independent Music Radio", and only this string, is the title.

Some tags have **attributes**, which are arguments listed inside an opening tag to modify the behavior of that tag or to attach relevant data to the tag. The first `<html>` tag listed above contains an attribute `lang` with a value `"en"` that specifies that this document contains HTML code in English.



### <a name="parse"></a> Parsing Raw HTML Using `BeautifulSoup()`
The `requests.get()` function only downloads the raw text of the HTML code, but it does not yet understand the logic and organization of the HTML code. Getting Python to register text as a particular coding standard is called **parsing** the code. We've parsed code into Python before with JSON data. We used `requests.get()` to download the JSON formatted data, but we needed `json.loads()` to parse the data in order to be able to navigate the branches of the JSON tree. 

There are two widely used Python libraries for parsing HTML data: `bs4` which contains the `BeautifulSoup()` function, and `selenium`. `BeautifulSoup()` works with raw text, but cannot access websites themselves (we use `requests.get()` for that). In order to access the data on a website, the data needs to be visible in the raw HTML that `requests.get()` returns. If there are measures taken by a website to hide that data, possibly by calling server-side Javascript to populate data fields, or by saving data as image files, then we won't be able to access the data with an HTML parser. `selenium` has more features to extract more complicated data and circumvent anti-scraping measures, such as taking a screenshot of the webpage in a browser and using optical character recognition (OCR) to pull data directly from the image. However, `selenium` requires each request to be loaded in a web browser, so it can be quite a bit slower than `BeautifulSoup()`. If you are interested in learning how to use `selenium`, see this guide: https://selenium-python.readthedocs.io/. Here we will be using `BeautifulSoup()`.

First I import the `BeautifulSoup()` function:

In [7]:
from bs4 import BeautifulSoup

To use it, we pass the `.text` attribute of the `requests.get()` output from https://spinitron.com/WNRN to `BeautifulSoup()` (which I saved as `r.text` above). This function can parse either HTML or XML code, so the second argument should specify HTML:

In [8]:
wnrn = BeautifulSoup(r.text, 'html')

Now that the https://spinitron.com/WNRN source code is registered as HTML code in Python, we can begin executing commands to navigate the organizational structure of the code and extract data. 

### <a name="tags"></a> Searching for HTML Tags and Extracting Data
While HTML is a coding language, it does not force coders to follow very strict templates. There's a lot of flexibility and creativity possible for HTML programmers, and as such, there is no one universal method for extracting data from HTML. The best approach is to open a browser window, navigate to the webpage you want to scrape, and "view page source". (Different web browsers have different ways to do that. On Mozilla Firefox, right click somewhere on the page other than an active link, and "view page source" should be an option.) The source will display the raw HTML code that generates the page. You will need to search through this code to find examples of the data points you intend to collect, possibly using control+F to search for specific values. Once you find the data you need, make note of the tags that surround the data and use the tools we will describe next to extract the data.

The parsable HTML `BeautifulSoup()` output, `wnrn`, has important methods and attributes that we will use to extract the data we want. First, we can use the name of a tag as an attribute to extract the first occurrence of that tag. Here we extract the first `<meta>` tag:

In [10]:
metatag = wnrn.meta
metatag

<meta charset="utf-8"/>

This tag stores its attributes as a list, so we can extract the value of an attribute by calling the name of that attribute as follows:

In [11]:
metatag['charset']

'utf-8'

If a tag has a navigable string, we can extract that with the `.string` attribute of a particular tag. For example, to extract the title, we start with the `<title>` tag:

In [12]:
titletag = wnrn.title
titletag

<title>WNRN – Independent Music Radio</title>

Then we extract the title as follows:

In [13]:
titletag.string

'WNRN – Independent Music Radio'

Our goal in this example is to extract the artist, song, album, and time played for every song played on WNRN. I look in the raw HTML source code for the first instance of an artist. These data are contained in the `<span>` tags:

In [14]:
spantag = wnrn.span
spantag

<span class="artist">Khruangbin &amp; Leon Bridges</span>

Calling one tag is not especially useful, because we generally want to extract all of the relevant data on a page. For that, we can use the `.find_next()` and `.find_all()` methods, both of which are very literal. The next `<span>` tag in the HTML code contains the song associated with the artist:

In [15]:
spantag.find_next()

<span class="song">Texas Sun</span>

And the next occurrence of `<span>` contains the album name (under `"release"`):

In [16]:
spantag.find_next().find_next()

<div class="info"><span class="release">Texas Sun</span></div>

To find all occurrences of the `<span>` tag, organized in a list, use `.find_all()` and provide the tag as the argument:

In [17]:
spanlist = wnrn.find_all("span")
spanlist

[<span class="artist">Khruangbin &amp; Leon Bridges</span>,
 <span class="song">Texas Sun</span>,
 <span class="release">Texas Sun</span>,
 <span class="artist">Talking Heads</span>,
 <span class="song">Burning Down The House</span>,
 <span class="release">Speaking In Tongues</span>,
 <span class="artist">Steve Earle</span>,
 <span class="song">You're Still Standin' There</span>,
 <span class="release">I Feel Alright</span>,
 <span class="artist">Rookie</span>,
 <span class="song">Sunglasses</span>,
 <span class="release">Rookie</span>,
 <span class="artist">Heartless Bastards</span>,
 <span class="song">Parted Ways</span>,
 <span class="release">Arrows</span>,
 <span class="artist">Leo Kottke</span>,
 <span class="song">Stolen</span>,
 <span class="release">Try And Stop Me</span>,
 <span class="artist">Stray Fossa</span>,
 <span class="song">It's Nothing</span>,
 <span class="release">(Single)</span>,
 <span class="artist">The Raconteurs</span>,
 <span class="song">Help Me Stranger</s

Notice that the HTML source code distinguishes between the three types of datapoint with different `class` values. To limit this list to just the artists, we can specify the `"artist"` class as a second argument of `.find_all()`:

In [18]:
artistlist = wnrn.find_all("span", "artist")
artistlist

[<span class="artist">Khruangbin &amp; Leon Bridges</span>,
 <span class="artist">Talking Heads</span>,
 <span class="artist">Steve Earle</span>,
 <span class="artist">Rookie</span>,
 <span class="artist">Heartless Bastards</span>,
 <span class="artist">Leo Kottke</span>,
 <span class="artist">Stray Fossa</span>,
 <span class="artist">The Raconteurs</span>,
 <span class="artist">My Morning Jacket</span>,
 <span class="artist">Drive-By Truckers</span>,
 <span class="artist">Boy and Bear</span>,
 <span class="artist">Andy Jenkins</span>]

Likewise we can create lists of the songs:

In [19]:
songlist = wnrn.find_all("span", "song")
songlist

[<span class="song">Texas Sun</span>,
 <span class="song">Burning Down The House</span>,
 <span class="song">You're Still Standin' There</span>,
 <span class="song">Sunglasses</span>,
 <span class="song">Parted Ways</span>,
 <span class="song">Stolen</span>,
 <span class="song">It's Nothing</span>,
 <span class="song">Help Me Stranger</span>,
 <span class="song">Compound Fracture</span>,
 <span class="song">Armageddon's Back in Town</span>,
 <span class="song">Southern Sun</span>,
 <span class="song">Dawn On</span>]

And a list for the albums:

In [20]:
albumlist = wnrn.find_all("span", "release")
albumlist

[<span class="release">Texas Sun</span>,
 <span class="release">Speaking In Tongues</span>,
 <span class="release">I Feel Alright</span>,
 <span class="release">Rookie</span>,
 <span class="release">Arrows</span>,
 <span class="release">Try And Stop Me</span>,
 <span class="release">(Single)</span>,
 <span class="release">Help Us Stranger</span>,
 <span class="release">The Waterfall</span>,
 <span class="release">The Unraveling</span>,
 <span class="release">Harlequin Dream</span>,
 <span class="release">The Garden Opens</span>]

Finally, we want to also extract the times each song was played. I look at the HTML code and find an example of the play time. These times are stored in the `<td>` tag with `class="spin-time"`. I create a list of these times:

In [21]:
timelist = wnrn.find_all("td", "spin-time")
timelist

[<td class="spin-time"><a href="/WNRN/pl/10646322/WNRN?sp=220561632">6:43 AM</a></td>,
 <td class="spin-time"><a href="/WNRN/pl/10646322/WNRN?sp=220561344">6:39 AM</a></td>,
 <td class="spin-time"><a href="/WNRN/pl/10646322/WNRN?sp=220561065">6:36 AM</a></td>,
 <td class="spin-time"><a href="/WNRN/pl/10646322/WNRN?sp=220560687">6:31 AM</a></td>,
 <td class="spin-time"><a href="/WNRN/pl/10646322/WNRN?sp=220560273">6:26 AM</a></td>,
 <td class="spin-time"><a href="/WNRN/pl/10646322/WNRN?sp=220560075">6:23 AM</a></td>,
 <td class="spin-time"><a href="/WNRN/pl/10646322/WNRN?sp=220559829">6:20 AM</a></td>,
 <td class="spin-time"><a href="/WNRN/pl/10646322/WNRN?sp=220559532">6:16 AM</a></td>,
 <td class="spin-time"><a href="/WNRN/pl/10646322/WNRN?sp=220559271">6:12 AM</a></td>,
 <td class="spin-time"><a href="/WNRN/pl/10646322/WNRN?sp=220559001">6:08 AM</a></td>,
 <td class="spin-time"><a href="/WNRN/pl/10646322/WNRN?sp=220558602">6:04 AM</a></td>,
 <td class="spin-time"><a href="/WNRN/pl/10

Sometimes the information we need exists in a particular tag, but only when a specific attribute is present. For example, in the WNRN playlist HTML there are many `<a>` tags, but only some of those tags include a `title` attribute. To extract all of the `<a>` tags with a `title` attribute, specify `title=True` in the call to `.find_all()`:

In [34]:
atags_title = wnrn.find_all("a", title=True)
print(atags_title[0:5]) # just show the first 6 elements

[<a class="buy-link" data-vendor="apple" href="#" target="_blank" title='View "Khruangbin &amp; Leon Bridges - Texas Sun" on Apple'><div alt='View "Khruangbin &amp; Leon Bridges - Texas Sun" on Apple' class="buy-icon buy-icon-apple"></div></a>, <a class="buy-link" data-vendor="amazon" href="#" target="_blank" title='View "Khruangbin &amp; Leon Bridges - Texas Sun" on Amazon'><div alt='View "Khruangbin &amp; Leon Bridges - Texas Sun" on Amazon' class="buy-icon buy-icon-amazon"></div></a>, <a class="buy-link" data-vendor="spotify" href="#" target="_blank" title='View "Khruangbin &amp; Leon Bridges - Texas Sun" on Spotify'><div alt='View "Khruangbin &amp; Leon Bridges - Texas Sun" on Spotify' class="buy-icon buy-icon-spotify"></div></a>, <a class="buy-link" data-vendor="apple" href="#" target="_blank" title='View "Talking Heads - Burning Down The House" on Apple'><div alt='View "Talking Heads - Burning Down The House" on Apple' class="buy-icon buy-icon-apple"></div></a>, <a class="buy-lin

### <a name="dataframe"></a> Constructing a Data Frame from HTML Data
Next we need to place these data into a clean data frame. For that, we will need to keep the valid data while dropping the HTML tags. We stored the tags with the artists, songs, albums, and times in separate lists. Every name is stored as a navigable string in the HTML tags, so to extract these names we need to loop across the elements of the list. The simplest loop for this task is called a **list comprehension**, which has the following syntax:

*newlist* `= [` *expression* `for` *item* `in` *oldlist* `if` *condition* `]`

In this syntax, we are creating a new list by iteratively performing operations on the elements of an existing list (*oldlist*). *item* is a token that we will use to represent one item of the existing list. *expression* is the same Python code we would use on a single element of the existing list, except we replace the name of the element with the token defined with *item*. Finally *condition* is an optional part of this code which sets a filter by which only certain elements of the old list are transformed and placed into the new list (there's an example of conditioning in a comprehension loop in the section on [spiders](#spider)).

For example, to extract the navigable string from every element of `artistlist`, we can set *item* to `a`, *expression* to `a.string`, and *list* to `artistlist`:

In [23]:
artists = [a.string for a in artistlist]
artists

['Khruangbin & Leon Bridges',
 'Talking Heads',
 'Steve Earle',
 'Rookie',
 'Heartless Bastards',
 'Leo Kottke',
 'Stray Fossa',
 'The Raconteurs',
 'My Morning Jacket',
 'Drive-By Truckers',
 'Boy and Bear',
 'Andy Jenkins']

Likewise, we extract the navigable strings for the songs, albums, and times:

In [24]:
songs = [a.string for a in songlist]
albums = [a.string for a in albumlist]
times = [a.string for a in timelist]

Finally, to construct a clean data frame, we create a dictionary that combines these lists and passes this dictionary to the `pd.DataFrame()` function:

In [25]:
mydict = {'time':times,
          'artist':artists,
         'song':songs,
         'album':albums}
wnrn_df = pd.DataFrame(mydict)
wnrn_df

time                     artist                         song  \
0   6:43 AM  Khruangbin & Leon Bridges                    Texas Sun   
1   6:39 AM              Talking Heads       Burning Down The House   
2   6:36 AM                Steve Earle  You're Still Standin' There   
3   6:31 AM                     Rookie                   Sunglasses   
4   6:26 AM         Heartless Bastards                  Parted Ways   
5   6:23 AM                 Leo Kottke                       Stolen   
6   6:20 AM                Stray Fossa                 It's Nothing   
7   6:16 AM             The Raconteurs             Help Me Stranger   
8   6:12 AM          My Morning Jacket            Compound Fracture   
9   6:08 AM          Drive-By Truckers    Armageddon's Back in Town   
10  6:04 AM               Boy and Bear                 Southern Sun   
11  6:00 AM               Andy Jenkins                      Dawn On   

                  album  
0             Texas Sun  
1   Speaking In Tongues  
2        I Feel Alright  
3                Rookie  
4                Arrows  
5       Try And Stop Me  
6              (Single)  
7      Help Us Stranger  
8         The Waterfall  
9        The Unraveling  
10      Harlequin Dream  
11     The Garden Opens

## <a name="spider"></a>Building a Spider
At the bottom of the WNRN playlist on https://spinitron.com/WNRN/ there are links to older song playlists. Let's extend our example by building a **spider** to capture the data that exists on these links as well. A spider is a web scraper that follows links on a page automatically and scrapes from those links as well. 

I look at the page source for these links, and find that they are contained in a `<div class="recent-playlists">` tag. I start by finding this tag. As there's only one occurrence, I can use `.find()` instead of `.find_all()`:

In [26]:
recent = wnrn.find("div", "recent-playlists")
recent

<div class="recent-playlists">
<h4>Recent</h4>
<div class="grid-view" id="w2">
<table class="table table-bordered table-narrow"><tbody>
<tr data-key="0"><td class="show-time">5:00 AM</td><td><strong><a href="/WNRN/pl/10646133/WNRN-4-4-20-5-00-AM">WNRN 4/4/20, 5:00 AM</a></strong> with <a href="/WNRN/dj/104061/WNRN">WNRN</a></td></tr>
<tr data-key="1"><td class="show-time">4:00 AM</td><td><strong><a href="/WNRN/pl/10646058/WNRN-4-4-20-4-02-AM">WNRN 4/4/20, 4:02 AM</a></strong> with <a href="/WNRN/dj/104061/WNRN">WNRN</a></td></tr>
<tr data-key="2"><td class="show-time">3:00 AM</td><td><strong><a href="/WNRN/pl/10645824/WNRN-4-4-20-3-01-AM">WNRN 4/4/20, 3:01 AM</a></strong> with <a href="/WNRN/dj/104061/WNRN">WNRN</a></td></tr>
<tr data-key="3"><td class="show-time">12:00 AM</td><td><strong><a href="/WNRN/pl/10645245/WNRN">WNRN</a></strong> (Music) with <a href="/WNRN/dj/104073/WNRN">WNRN</a></td></tr>
<tr data-key="4"><td class="show-time">8:00 PM</td><td><strong><a href="/WNRN/pl/10644

Notice that all of the addresses we need are contained in `<a>` tags. We can extract these `<a>` tags with `.find_all()`:

In [27]:
recent_atags = recent.find_all("a")
recent_atags

[<a href="/WNRN/pl/10646133/WNRN-4-4-20-5-00-AM">WNRN 4/4/20, 5:00 AM</a>,
 <a href="/WNRN/dj/104061/WNRN">WNRN</a>,
 <a href="/WNRN/pl/10646058/WNRN-4-4-20-4-02-AM">WNRN 4/4/20, 4:02 AM</a>,
 <a href="/WNRN/dj/104061/WNRN">WNRN</a>,
 <a href="/WNRN/pl/10645824/WNRN-4-4-20-3-01-AM">WNRN 4/4/20, 3:01 AM</a>,
 <a href="/WNRN/dj/104061/WNRN">WNRN</a>,
 <a href="/WNRN/pl/10645245/WNRN">WNRN</a>,
 <a href="/WNRN/dj/104073/WNRN">WNRN</a>,
 <a href="/WNRN/pl/10644264/WNRN">WNRN</a>]

The resulting list contains the web endpoints we need, and also some web endpoints we don't need: we want the URLs that contain the string `/pl/` as these are playlists, and we want to exclude the URLs that contain the string `/dj/` as these pages refer to a particular DJ. We need a comprehension loop that loops across these elements, extracts the `href` attribute of the entries that include `/pl/`, and ignore the entries that include `/dj/`. We again use this syntax:

*newlist* `= [` *expression* `for` *item* `in` *oldlist* `if` *condition* `]`

In this case:

* *newlist* is a list containing the URLs we want to direct our spider to. I call it `urls`.
* *item* is one element of `recent_atags`, which I will call `pl`.
* *expression* is code that extracts the web address from the `href` attribute of the `<a>` tag, so here the code would be `pl['href']`.
* Finally, *condition* is a logical statement that should be `True` if the web address contains `/pl/` and `False` if the web address contains `/dj/`. Here, the conditional statement should be `if "/pl/" in pl['href']`. This code will look for the string `"/pl/"` inside the string called by `pl['href']` and return `True` or `False` depending on whether this string is found.

Putting all this syntax together gives us our list of playlist URLs:

In [28]:
wnrn_url = [pl['href'] for pl in recent_atags if "/pl/" in pl['href']]
wnrn_url

['/WNRN/pl/10646133/WNRN-4-4-20-5-00-AM',
 '/WNRN/pl/10646058/WNRN-4-4-20-4-02-AM',
 '/WNRN/pl/10645824/WNRN-4-4-20-3-01-AM',
 '/WNRN/pl/10645245/WNRN',
 '/WNRN/pl/10644264/WNRN']

First, we need to collect all of the code we created above to extract the artist, song, album, and play times from the HTML code. We define a function that does all of this work. We specify one argument for this function, the URL, so that all the function needs is the URL and it can output a clean dataframe. I name the function `wnrn_spider()`:

In [29]:
def wnrn_spider(url):
    """Perform web scraping for any WNRN playlist given the available link"""
    
    headers = {'user-agent': 'Kropko class example (jkropko@virginia.edu)'}
    r = requests.get(url, headers=headers)
    wnrn = BeautifulSoup(r.text, 'html')
    
    artistlist = wnrn.find_all("span", "artist")
    songlist = wnrn.find_all("span", "song")
    albumlist = wnrn.find_all("span", "release")
    timelist = wnrn.find_all("td", "spin-time")
    
    artists = [a.string for a in artistlist]
    songs = [a.string for a in songlist]
    albums = [a.string for a in albumlist]
    times = [a.string for a in timelist]
    
    mydict = {'time':times, 'artist':artists, 'song':songs, 'album':albums}
    wnrn_df = pd.DataFrame(mydict)
    
    return wnrn_df


We can pass any of the URLs we collected to our function and get the other playlists. We will have to add the domain "https://spinitron.com" to the beginning of each of the URLs we collected:

In [30]:
wnrn2 = wnrn_spider('https://spinitron.com/' + wnrn_url[0])
wnrn2

time                       artist                 song  \
0   5:00 AM               Mumford & Sons        Guiding Light   
1   5:03 AM     The Tallest Man on Earth                 1904   
2   5:07 AM                         Beck  Everlasting Nothing   
3   5:12 AM                Caroline Rose  Feel the Way I Want   
4   5:16 AM               Marc Broussard                 Home   
5   5:23 AM                  Norah Jones            I'm Alive   
6   5:28 AM         Tedeschi Trucks Band               Anyhow   
7   5:32 AM               Anders Osborne              Running   
8   5:38 AM                    Nada Surf         So Much Love   
9   5:41 AM                  Andrew Bird             Sisyphus   
10  5:45 AM               Regina Spektor             Fidelity   
11  5:49 AM                  Marcus King             The Well   
12  5:52 AM  Jason Isbell & the 400 Unit            Be Afraid   
13  5:56 AM                  Beach House               Norway   

                       album  
0                      Delta  
1     There's No Leaving Now  
2                 Hyperspace  
3                  Superstar  
4                   Carencro  
5   Pick Me up Off the Floor  
6              Let Me Get By  
7       Buddha and the Blues  
8         Never Not Together  
9         My Finest Work Yet  
10             Begin To Hope  
11                 El Dorado  
12                  Reunions  
13                Teen Dream

Our goal here is to loop across all the URLs we collected, extract the data in a clean data frame, and append these data frames together to construct a longer playlist. To do that, we will use a `for` loop, which has the following syntax:
```
for index in list:
    expressions
```
This syntax is similar to the syntax we used to build a comprehension loop. `list` is an existing list, and `index` stands in for one element of this list. For each element of the list, we execute the code contained in `expressions`, which can use the `index`.

For our spider, we will use the following steps:

1. We take the data we already scraped from https://spinitron.com/WNRN (saved as `wnrn_df`) and clone it as a new variable named `wnrn_total_playlist`. It is important that we make a copy, and that we do not overwrite `wnrn_df`. We will be repeatedly saving over `wnrn_total_playlist` within the loop, and if we do not overwrite `wnrn_df`, it gives us a stable data frame to return to as a starting point if we need to rerun this loop. 

2. We use a `for` loop to loop across all the web addresses inside `wnrn_url`.

3. In the `for` loop, we use the `wnrn_spider()` function to extract the playlist data from each of the URLs inside `wnrn_url`.

4. In the `for` loop, we use the `.append()` method to attach the new data to the bottom of the existing data, matching corresponding columns.

The code is as follows:

In [31]:
wnrn_total_playlist = wnrn_df 
for w in wnrn_url:
    moredata = wnrn_spider('https://spinitron.com/' + w) 
    wnrn_total_playlist = wnrn_total_playlist.append(moredata)

We now have a data frame that combines all of the playlists on https://spinitron.com/WNRN and on the playlists linked to under "Recent":

In [32]:
wnrn_total_playlist

time                     artist                         song  \
0    6:43 AM  Khruangbin & Leon Bridges                    Texas Sun   
1    6:39 AM              Talking Heads       Burning Down The House   
2    6:36 AM                Steve Earle  You're Still Standin' There   
3    6:31 AM                     Rookie                   Sunglasses   
4    6:26 AM         Heartless Bastards                  Parted Ways   
..       ...                        ...                          ...   
53  11:43 PM                Waxahatchee                       Lilacs   
54  11:47 PM         Nathaniel Rateliff       And It's Still Alright   
55  11:51 PM                      Spoon                 Hot Thoughts   
56  11:55 PM         Alejandro Escovedo             Sister Lost Soul   
57  11:58 PM                    Vagabon                Water Me Down   

                     album  
0                Texas Sun  
1      Speaking In Tongues  
2           I Feel Alright  
3                   Rookie  
4                   Arrows  
..                     ...  
53             Saint Cloud  
54  And It's Still Alright  
55            Hot Thoughts  
56             Real Animal  
57                 Vagabon  

[162 rows x 4 columns]